# Matching, Поиск ближайшего соседа  
В данном проекте было необходимо разработать алгоритм который для всех товаров из списка может предложить несколько вариантов наиболее похожих товаров из базы данных. 
В моём распоряжении находилась база данных с 3 миллионами товаров и 72 признаками для каждого, тренировочный набор данных (100000 строк) и данные для валидации (100000 строк). 
 Заказчик предварительно зашифровал все данные, поэтому анализ признаков не представлялся возможным.

## Данные

- *base.csv* - анонимизированный набор товаров. Каждый товар представлен как уникальный id (0-base, 1-base, 2-base) и вектор признаков размерностью 72.
- *target.csv -* обучающий датасет. Каждая строчка - один товар, для которого известен уникальный id (0-query, 1-query, …) , вектор признаков И id товара из *base.csv*, который максимально похож на него (по мнению экспертов).
- *validation.csv* - датасет с товарами (уникальный id и вектор признаков), для которых надо найти наиболее близкие товары из *base.csv*
- *validation_answer.csv* - правильные ответы к предыдущему файлу.  

Задача была разбита на два этапа. В первом была применена библиотека от разработанная Facebook -"Facebook AI Similarity Search". Необходимо было выбрать n кандидатов для каждого запроса из target и на втором
этапе с помощью модели CatBoostClassifier проранжировать их.На первом этапе (Файл Matching.ipynb) было проанализировано несколько вариантов индексации и квантования признаков:инвертированный индекс (IVF), product quantization (PQ), инвертированный мульти индекс (IMI), их различные комбинации,
а также методы Refine, RFlat и OPQ. Наилучший результат в index_factory показала строка "OPQ32,IMI2x8,PQ32" со значением метрики accuracy@5 = 82 для 200 ближайших соседей и 84.5 для 500 и nprobe = 10000. Хочется отметить несколько интересных фактов:

    1) Применение метода масштабирования признаков StandardScaler в последствии даёт меньший результат, чем MinMaxScaler  
    2) Для IVFPQ можно разбить вектор признаков на две части и для каждой обучить свою модель faiss. После объединения предсказаний и удаления дубликатов, можно получить метрику на 1% выше, чем для одной модели faiss для одинакового выходного количества соседей. Данный метод плохо работает для IVFFlat и HNSWFlat  
    3) Некоторые признаки сильно ухудшают работу faiss, поэтому первый этап проводился без них  

# Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import time

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

import random
import faiss
from faiss import write_index, read_index

from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

C:\ProgramData\Anaconda3\envs\practicum\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Загружаем все данные для работы
def upload_data():
    df_base = pd.read_csv("data/base.csv", index_col=0)
    df_train = pd.read_csv("data/train.csv", index_col=0)
    df_validation = pd.read_csv("data/validation.csv", index_col=0)
    validation_targets = pd.read_csv("data/validation_answer.csv", index_col=0)
    
    train_targets = df_train["Target"]
    df_train.drop("Target", axis=1, inplace=True)
    
    scale = MinMaxScaler(feature_range = (-2, 2))
    base_scaled = pd.DataFrame(scale.fit_transform(df_base), columns=df_base.columns, index=df_base.index)
    train_scaled = pd.DataFrame(scale.transform(df_train), columns=df_train.columns, index=df_train.index)
    validation_scaled = pd.DataFrame(scale.transform(df_validation), columns=df_validation.columns, index=df_validation.index)
    
    base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
    
    return base_scaled, train_scaled, validation_scaled, train_targets.values.tolist(), validation_targets['Expected'].values.tolist(), base_index

In [48]:
#Метрика точности
def accuracy_n(target: list, ans: list()):
    acc = 0
    for target, el in zip(target, ans):
        acc += int(target in [base_index[r] for r in el])
    print("Точность = ", 100 * acc / len(ans))
    return 100 * acc / len(ans)

In [65]:
#Первый этап: отбор кандидатов для ранжирования
def search_in_index(index, data: np.ndarray, out: int, target: list):
    bad_features= ['6', '21', '25', '44', '59', '70', '65', '33']
    
    if type(data) == pd.DataFrame:
        data = data.drop(bad_features, axis=1)
        data = data.values
    start_time = time.time()
    vecs, idx = index.search(np.ascontiguousarray(data).astype('float32'), out)
    print(f"Время поиска кандидатов (выдача = {out}) = {round(time.time() - start_time, 3)} секунд")
    accuracy_n(target, idx)
    return vecs, idx

In [6]:
#Загрузка индексов для последующей работы search_in_index
def get_index(find_data, target, nprobe=10000, number_of_neighbors=100):
    index = read_index("OPQ32,IMI2x8,PQ32_nprobe10000_parts1_length64_out_one200_part.index")
    index.nprobe = 10000
    vecs, idx = search_in_index(index, find_data, number_of_neighbors, target)
    return vecs, idx

In [5]:
#Формаирование признаков для модели CatBoost
def generate_dataset(base, query, idx, vecs, base_index, number_of_neighbors, query_target):
    faiss_index = np.array(idx).flatten()
    decode_index = np.array([[base_index[r] for r in el] for el in idx])
    faiss_rank = np.array([[r+1 for r in range(len(el))] for el in idx]).flatten()
    idx = idx.flatten()
    
    base_features = base.loc[decode_index.flatten()].values
    query_features = np.repeat(query.values, number_of_neighbors, axis=0)
    vecs = np.array(vecs).reshape(-1, 1)
    
    target = []
    for t, el in zip(query_target, decode_index):
        target_query = []
        for i in el:
            if i != -1 and t == i:
                target_query.append(1)
            else:
                target_query.append(0)  
        target.append(target_query)
        
    target = np.array(target).flatten()
    
    features = np.column_stack((vecs, query_features, base_features, np.sqrt(np.square(query_features - base_features))))
    return features, target

In [11]:
number_of_neighbors = 120
nprob = 10000 #Всего их 2**18=262144

In [12]:
base, train, valid, train_target, valid_target, base_index = upload_data()

In [13]:
vecs, idx = get_index(train, train_target, nprob, number_of_neighbors)

Время поиска 120 кандидатов = 248.49448609352112 секунд
Точность =  80.931


In [14]:
features, target = generate_dataset(base, train, idx, vecs, base_index, number_of_neighbors, train_target)

# Обучение

In [16]:
#Обучение модели
def fit_model(features,  target):
    X_train, X_test, y_train, y_test = train_test_split(features,  target, 
                                                    test_size=0.05, random_state=42, shuffle=True)
    
    #classes = np.unique(y_train)
    #weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
    #class_weights = dict(zip(classes, weights))
    
    class_weights = {0: 1, 1: 100}
    #print(class_weights)
    params = {
        'max_depth': 8,
        'n_estimators': 850,
        'learning_rate': 0.1,
        'thread_count': -1,
        'random_state': 42,
        'verbose': 50,
        'class_weights':  class_weights,
        'custom_loss': ['AUC', 'Accuracy']
    }
    
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_test,  y_test))
    
    model.save_model('catboost_model_final',
           format="cbm",
           export_parameters=None,
           pool=None)
    

In [17]:
fit_model(features,  target)

0:	learn: 0.5013310	test: 0.5016736	best: 0.5016736 (0)	total: 3.97s	remaining: 56m 12s
50:	learn: 0.0971326	test: 0.0992343	best: 0.0992343 (50)	total: 2m 4s	remaining: 32m 35s
100:	learn: 0.0862218	test: 0.0908612	best: 0.0908612 (100)	total: 4m 7s	remaining: 30m 37s
150:	learn: 0.0795361	test: 0.0876097	best: 0.0875867 (149)	total: 6m 8s	remaining: 28m 26s
200:	learn: 0.0730327	test: 0.0835361	best: 0.0835361 (200)	total: 8m	remaining: 25m 51s
250:	learn: 0.0680262	test: 0.0816328	best: 0.0816328 (250)	total: 9m 51s	remaining: 23m 31s
300:	learn: 0.0640902	test: 0.0803301	best: 0.0803194 (299)	total: 11m 36s	remaining: 21m 10s
350:	learn: 0.0606478	test: 0.0792059	best: 0.0791913 (349)	total: 13m 20s	remaining: 18m 57s
400:	learn: 0.0577873	test: 0.0786697	best: 0.0786697 (400)	total: 15m 3s	remaining: 16m 51s
450:	learn: 0.0548320	test: 0.0779374	best: 0.0778855 (447)	total: 16m 48s	remaining: 14m 52s
500:	learn: 0.0521914	test: 0.0773270	best: 0.0773201 (497)	total: 18m 34s	remain

# Валидация модели

In [11]:
#Если нет необходимости в данных для обучения, то удаляем их
del train, train_target

In [18]:
#Загрудаем обученную модель CatBoost
def load_model():
    model = CatBoostClassifier()
    model.load_model('catboost_model_final')
    return model

model = load_model()

In [83]:
"""
sort_pred - сортирует отранжированную матрицу для отбора лучших кандидатов
validation - Своего рада валидация батчами для экономии ОЗУ
"""

number_of_neighbors = 1000
nprob = 10000 #Всешо их 2**18=262144

def sort_pred(pred, index):
    a = np.array([pred, index])
    for i in range(len(a[0])):
        b = a[:,i,:][:, :].argsort(axis=1)
        b = b[0].tolist()
        a[:,i,:] = a[:,i,b]
        a[:,i,:] = a[:,i,::-1]
    return a

#start, end - начало и конец среза в valid
def validation(start, end):
    vecs_valid, idx_valid = get_index(valid[start:end], valid_target[start:end], nprob, number_of_neighbors)
    
    features_valid, target_valid = generate_dataset(base, valid[start:end], idx_valid, vecs_valid, 
                                                base_index, number_of_neighbors, valid_target[start:end])
    pred = model.predict_proba(features_valid)[:,1]
    pred = pred.reshape((-1, number_of_neighbors))
    arr = sort_pred(pred, idx_valid)
    acc = accuracy_n(valid_target[start:end], arr[:,:,:5][1].astype(int).tolist())
    return acc

Цикл ниже возвращает значение метрики для небольших частей данных валидации

In [84]:
total_acc = []
for i in range(1):
    print(f"Часть {i+1}")
    total_acc.append(validation(10000*i, 10000*(i+1)))
    print()
    
print(f"Суммарная точность на валидации {np.mean(total_acc)}")

Часть 1
Время поиска кандидатов (выдача = 1000) = 24.818 секунд
Точность =  85.06
Точность =  82.63

Суммарная точность на валидации 82.63


# Итоги

Был разработана двухэтапная модель для определения ближайших соседей "неизвестного" интернет магазина. Результаты тестирования модели представленны в таблице:

| Количество соседей, n | Accuracy@n после первого этапа | Время выполнения валидации | Accuracy@5 |
|:----------------:|:----------------:|:---------:|:----------------:|
| 1000 (на 10% данных) | 85.06 | 10m 1s | 82.63 |
| 500 | 84.073 | 48m 35s | 81.713 |
| 200 | 82.115| 22m 9s | 80.148 |
| 50 | 78.67| 8m 44 | 77.4 |


Время выполнения первого этапа не зависит от количества соседей и равно примерно 250 секунд. Самый большой вклад во время работы запроса - ранжирование CatBoost. 

Если cделать 1000 запросов по одной строке из данных valid, то при n = 500 суммарное время выполнения = 2m 45s, или 0.17 секунды на запрос
